# Día 7 Parte 1: Introducción a APIs con Python

## Ejemplos

**Ejemplo 7.1: Ejemplo con API Real: OpenWeatherMap**

**Descripción:**

Vamos a conectarnos con la API de clima de OpenWeatherMap para consultar el clima de una ciudad.

**Requisitos:**

Necesitas una cuenta gratuita en [https://openweathermap.org/api](https://openweathermap.org/api) y obtener una **API Key**.

In [ ]:
import requests

API_KEY = "API-Key"  # Reemplázala con tu clave real
CITY = "Lima"

url = f"http://api.weatherapi.com/v1/current.json?key={API_KEY}&q={CITY}&aqi=no"

response = requests.get(url)
data = response.json()

print("Ciudad:", data["location"]["name"])
print("País:", data["location"]["country"])
print("Temperatura (C):", data["current"]["temp_c"])
print("Condición:", data["current"]["condition"]["text"])


In [ ]:
import requests
import pandas as pd

API_KEY = "API-Key"
CITY = "Lima"

url = f"http://api.weatherapi.com/v1/forecast.json?key={API_KEY}&q={CITY}&days=7"

response = requests.get(url).json()

fechas = []
maximas = []
minimas = []

for d in response["forecast"]["forecastday"]:
    fechas.append(d["date"])
    maximas.append(d["day"]["maxtemp_c"])
    minimas.append(d["day"]["mintemp_c"])

df = pd.DataFrame({
    "fecha": fechas,
    "temp_max": maximas,
    "temp_min": minimas
})

print(df)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df["fecha"], df["temp_max"])
plt.plot(df["fecha"], df["temp_min"])
plt.xticks(rotation=45)
plt.show()


**Ejemplo 7.2: Precios de Criptomonedas con CoinGecko**

CoinGecko ofrece datos en tiempo real sin necesidad de autenticación.

In [ ]:
import requests
import matplotlib.pyplot as plt

criptos = ["bitcoin", "ethereum", "litecoin"]
precios = []

for cripto in criptos:
    url = "https://api.coingecko.com/api/v3/simple/price"
    params = {"ids": cripto, "vs_currencies": "usd"}
    r = requests.get(url, params=params)
    precios.append(r.json()[cripto]['usd'])

# Gráfico de barras
plt.bar(criptos, precios)
plt.title("Precios actuales de criptomonedas")
plt.ylabel("Precio en USD")
plt.show()

## Ejemplos adicionales

**Ejemplo 7.3: Descargar y graficar el balance eléctrico diario**

Visualizar la evolución diaria del balance eléctrico de España en un mes específico.

API usada:

`https://apidatos.ree.es/es/datos/balance/balance-electrico`

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json  # Asegúrate de ejecutar esta línea

url = "https://apidatos.ree.es/es/datos/balance/balance-electrico"
params = {
    "start_date": "2025-01-01T00:00",
    "end_date": "2025-01-31T23:59",
    "time_trunc": "day",
    "item": "generacion"
}
headers = {
    "Accept": "application/json"
}

response = requests.get(url, params=params, headers=headers)
data = response.json()

# Inspeccionar el JSON completo
print(json.dumps(data, indent=2, ensure_ascii=False))



In [ ]:
print(data['included'][0]['attributes'].keys())


In [ ]:
# Lista de fuentes de generación
fuentes = data['included']
fuentes

In [ ]:
# Accedemos a la fuente "Hidráulica"
fuente_hidraulica = data['included'][0]['attributes']['content'][0]  # 0 = hidráulica
nombre = fuente_hidraulica['attributes']['title']
valores = fuente_hidraulica['attributes']['values']

# Crear DataFrame
df = pd.DataFrame(valores)
df['datetime'] = pd.to_datetime(df['datetime'])
df.rename(columns={'value': 'GWh'}, inplace=True)

# Graficar
plt.figure(figsize=(12,6))
plt.plot(df['datetime'], df['GWh'], marker='o')
plt.title(f"{nombre} - Generación diaria (GWh)")
plt.xlabel("Fecha")
plt.ylabel("Energía (GWh)")
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(12, 6))

# Todas las fuentes:
for fuente in data['included'][0]['attributes']['content']:
    nombre = fuente['attributes']['title']
    valores = fuente['attributes']['values']
    df = pd.DataFrame(valores)
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.rename(columns={'value': nombre}, inplace=True)
    plt.plot(df['datetime'], df[nombre], label=nombre)

plt.legend()
plt.title("Generación diaria por fuente renovable (GWh)")
plt.xlabel("Fecha")
plt.ylabel("Energía (GWh)")
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
import requests
import matplotlib.pyplot as plt

url = "https://apidatos.ree.es/es/datos/generacion/estructura-generacion"
params = {
    "start_date": "2023-01-01T00:00",
    "end_date": "2023-01-31T23:59",
    "time_trunc": "month"
}
headers = {"Accept": "application/json"}

r = requests.get(url, params=params, headers=headers)
datos = r.json()

# Datos por tecnología
techs = datos['included']

# Lista de tecnologías renovables
renovables = ['hidráulica', 'eólica', 'solar fotovoltaica', 'solar térmica', 'biomasa']

ren = 0
no_ren = 0

for t in techs:
    nombre = t['attributes']['title'].lower()
    valores = t['attributes'].get('values', [])
    if valores:  # Verifica que la lista no esté vacía
        energia = valores[0]['value']
        if nombre in renovables:
            ren += energia
        else:
            no_ren += energia

# Gráfico
plt.pie([ren, no_ren], labels=["Renovable", "No Renovable"], autopct="%1.1f%%")
plt.title("Participación renovable en enero 2023")
plt.axis("equal")
plt.show()


**Ejemplo 7.4: Descargar y guardar datos para uso posterior**

In [ ]:
import os
os.makedirs("datos_guardados", exist_ok=True)

with open("datos_guardados/balance_enero.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

df.to_csv("datos_guardados/balance_enero.csv", index=False)

Esto permite:

* Automatizar informes
* Hacer análisis offline
* Compartir resultados con otros equipos

## Ejercicios

**Ejercicio 7.1: Clima**

Consulta el clima actual de tres ciudades distintas usando la API de OpenWeatherMap.

In [ ]:
# Espacio para resolver el ejercicio

**Ejercicio 7.2: Noticias (opcional)**

Conéctate a la API de [NewsAPI](https://newsapi.org/) y muestra los titulares relacionados con "energía".

In [ ]:
!pip install newsapi-python

In [ ]:
# Espacio para resolver el ejercicio

In [ ]:
from newsapi import NewsApiClient

newsapi = NewsApiClient(api_key='API-Key')  # Reemplázala con tu clave real

# /v2/top-headlines
top_headlines = newsapi.get_top_headlines(sources='bbc-news')

top_headlines

In [ ]:
# Lo anterior regresa en formato JSON (diccionario). Para convertirlo:

for article in top_headlines['articles']:
    titulo = article['title']
    enlace = article['url']
    print(f"{titulo}\n{enlace}\n")


In [ ]:
# O bien a dataframe:
import pandas as pd

df = pd.DataFrame(top_headlines['articles'])
print(df[['title', 'description', 'url', 'publishedAt']])


In [ ]:
all_articles = newsapi.get_everything(q='electricity',
                                      sources='abc-news,bbc-news,the-verge, bloomberg,business-insider',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param='2025-06-20',
                                      to='2025-07-10',
                                      language='en',
                                      sort_by='relevancy',
                                      page=1)

all_articles

In [ ]:
for article in all_articles['articles']:
    title = article['title']
    description = article['description']
    url = article['url']
    print(f"{title}\n{description}\n{url}\n---\n")

In [ ]:
import pandas as pd

df = pd.DataFrame(all_articles['articles'])
print(df[['title', 'description', 'url', 'publishedAt']])

# Adicional: Programar la ejecución diaria de forma automática.

```bash
import schedule
import time

def tarea():
    # Aquí podemos poner el código de la API y la graficación.
    pass

schedule.every().day.at("07:00").do(tarea)

while True:
    schedule.run_pending()
    time.sleep(60)
```